In [1]:
import pandas as pd

### ad

In [2]:
df_tr_ad = pd.read_csv('../data/raw/train/ad.csv', sep=',')
df_tr_ad.shape

(2481135, 6)

In [3]:
ad_cols = list(df_tr_ad.columns)

In [4]:
df_tr_ad.head()

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,4,4,\N,5,108,202
2,7,7,\N,5,148,297
3,8,8,\N,5,713,213
4,9,9,\N,5,695,213


In [5]:
df_te_ad = pd.read_csv('../data/raw/test/ad.csv', sep=',')
df_te_ad.shape

(2618159, 6)

In [6]:
df_te_ad.head()

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,3,3,\N,5,598,165
2,4,4,\N,5,108,202
3,5,5,\N,5,109,6
4,6,6,\N,5,400,6


In [7]:
def diff_dist(df_tr, df_te, sel_cols):
    res = dict()
    for col in sel_cols:
        print('current col is: ', col)
#         if col in ['product_id', 'industry']:
#             tr_set = set(df_tr[df_tr[col] != '\\N'][col])
#             te_set = set(df_te[df_te[col] != '\\N'][col])
#         else:
        tr_set = set(df_tr[col])
        te_set = set(df_te[col])
        inter_set = tr_set & te_set
        print('num of vals in train data is: ', len(tr_set))
        print('num of vals in test data is: ', len(te_set))
        print('num of vals in inter data is: ', len(inter_set))
        print('rate of inter&train data is: ', len(inter_set) / len(tr_set))
        print('rate of inter&test data is: ', len(inter_set) / len(te_set))
        print('\n')
        res[col] = pd.DataFrame(data=list(inter_set), columns=[col])
    return res

In [8]:
res_dict = diff_dist(df_tr_ad, df_te_ad, ad_cols)

current col is:  creative_id
num of vals in train data is:  2481135
num of vals in test data is:  2618159
num of vals in inter data is:  1686522
rate of inter&train data is:  0.6797381037307523
rate of inter&test data is:  0.6441633223956222


current col is:  ad_id
num of vals in train data is:  2264190
num of vals in test data is:  2379475
num of vals in inter data is:  1616305
rate of inter&train data is:  0.7138557276553646
rate of inter&test data is:  0.6792695867785962


current col is:  product_id
num of vals in train data is:  33273
num of vals in test data is:  34111
num of vals in inter data is:  28327
rate of inter&train data is:  0.8513509452108317
rate of inter&test data is:  0.8304359297587288


current col is:  product_category
num of vals in train data is:  18
num of vals in test data is:  18
num of vals in inter data is:  18
rate of inter&train data is:  1.0
rate of inter&test data is:  1.0


current col is:  advertiser_id
num of vals in train data is:  52090
num of va

### click

In [9]:
def gene_df(df_ad, click_path):
    df_click = pd.read_csv(click_path, sep=',')
    print('df_click shape is: ', df_click.shape)
    df_merge = pd.merge(df_click, df_ad, on='creative_id', how='left')
    print('df_merge shape is: ', df_merge.shape)
    del df_click
    return df_merge

In [10]:
df_tr_click = gene_df(df_tr_ad, '../data/raw/train/click_log.csv')
df_tr_click.head()

df_click shape is:  (30082771, 4)
df_merge shape is:  (30082771, 9)


,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,9,30920,567330,1,504423,30673,3,32638,319
1,65,30920,3072255,1,2642300,1261,2,6783,6
2,56,30920,2361327,1,2035918,1261,2,6783,6
3,6,309204,325532,1,292523,27081,3,32066,242
4,59,309204,2746730,1,2362208,\N,18,14682,88


In [11]:
df_te_click = gene_df(df_te_ad, '../data/raw/test/click_log.csv')
df_te_click.head()

df_click shape is:  (33585512, 4)
df_merge shape is:  (33585512, 9)


,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry
0,20,3131989,645764,1,573314,58,2,14689,6
1,20,3131989,1027422,1,902764,129,2,42272,6
2,20,3131989,1106443,1,970829,2171,2,37513,322
3,20,3131989,629802,1,559183,\N,18,14678,26
4,59,3131989,2839769,1,2441288,129,2,35328,6


In [12]:
len(set(df_te_click['user_id']))

1000000

In [13]:
def diff_dist2(df_tr, df_te, sel_cols, res_dict):
    for col in sel_cols:
        print('current col is: ', col)
        df_col = res_dict[col]
        df_tr_cover = pd.merge(df_tr[[col]], df_col, on=col)
        df_te_cover = pd.merge(df_te[[col]], df_col, on=col)
        print('df_tr cover is: ', df_tr_cover.shape[0] / df_tr.shape[0])
        print('df_te cover is: ', df_te_cover.shape[0] / df_te.shape[0])
    return

In [14]:
diff_dist2(df_tr_click, df_te_click, ad_cols, res_dict)

current col is:  creative_id
df_tr cover is:  0.9654974603237182
df_te cover is:  0.9620350286754599
current col is:  ad_id
df_tr cover is:  0.9710756033744364
df_te cover is:  0.9678852297978962
current col is:  product_id
df_tr cover is:  0.9997614913865481
df_te cover is:  0.9997299430778367
current col is:  product_category
df_tr cover is:  1.0
df_te cover is:  1.0
current col is:  advertiser_id
df_tr cover is:  0.9997522502165774
df_te cover is:  0.9997235712827602
current col is:  industry
df_tr cover is:  0.9999997008254327
df_te cover is:  0.9999997022525665
